In [8]:
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd

covid_dataset = pd.read_csv('google_new.csv', na_values=['NA'])
covid_dataset.head()


,iso,country,date,grocery_pharmacy,parks,residential,retail_recreation,transit_stations,workplaces,total_cases,fatalities
0,AR,Argentina,2020-02-23,8.185,17.563,0.437,13.644,5.334,-2.486,0,0
1,AR,Argentina,2020-02-24,-15.875,25.605,6.321,-9.973,-26.785,-53.687,0,0
2,AR,Argentina,2020-02-25,-17.135,1.325,6.895,-19.717,-28.485,-50.053,0,0
3,AR,Argentina,2020-02-26,2.304,5.399,-2.273,0.845,9.461,12.584,0,0
4,AR,Argentina,2020-02-27,-0.404,-5.267,-0.493,-0.584,7.983,12.407,0,0


In [7]:
covid_dataset.describe()

,grocery_pharmacy,parks,residential,retail_recreation,transit_stations,workplaces,total_cases,fatalities
count,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000,817.000000
mean,-8.229424,-13.394035,9.545845,-26.205257,-28.370122,-19.126192,10699.773562,566.007344
std,21.313369,32.891636,10.545377,30.992728,30.211025,25.983021,31656.464952,2008.520488
min,-84.798000,-91.456000,-3.647000,-95.158000,-88.956000,-79.293000,0.000000,0.000000
25%,-16.578000,-32.841000,0.744000,-49.975000,-54.145000,-40.582000,27.000000,0.000000
50%,0.088000,-6.498000,5.912000,-14.427000,-19.502000,-8.395000,648.000000,6.000000
75%,4.838000,6.961000,16.695000,-0.632000,-0.970000,1.037000,4585.000000,79.000000
max,59.474000,75.002000,40.343000,19.699000,16.822000,21.347000,336912.000000,15887.000000
